# Biol 359A | Write equations for diagrams
### Spring 2025, Week 7
Objectives:
- Convert diagrams to equations
- Simulate signal transduction systems with differential equations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
from ipywidgets import interact, interactive, fixed, FloatSlider, HBox, VBox, Layout, Button
from IPython.display import display, clear_output

## Write equations for diagrams - Signal transduction systems

In [ ]:
def solve_signal_transduction_odes(
    t_max=10.0, 
    X_init=28, XE_init=0.1, XP_init=0.2, XPE_init=0.01, XPP_init=0.02,
    E_init=0.01, E_step=10,
    k1=1, k2=0.1, k3=4, k4=1, k5=0.1, k6=4, k7=2, k8=2
):
    """
    Solves the system of ODEs for the signal transduction model
    """
    # Define the ODE system
    def ode_system(t, y):
        X, XE, XP, XPE, XPP, E = y
        
        # Handle the step change in E at t = 1
        if t >= 1:
            E = E_step
        
        # System of ODEs
        dXdt = -k1 * X * E + k2 * XE + k7 * XP
        dXEdt = k1 * X * E - (k2 + k3) * XE
        dXPdt = k3 * XE - k7 * XP - k4 * XP * E + k5 * XPE + k8 * XPP
        dXPEdt = k4 * XP * E - (k5 + k6) * XPE
        dXPPdt = k6 * XPE - k8 * XPP
        dEdt = 0  # E is controlled externally
        
        return [dXdt, dXEdt, dXPdt, dXPEdt, dXPPdt, dEdt]
    
    # Initial conditions
    y0 = [X_init, XE_init, XP_init, XPE_init, XPP_init, E_init]
    
    # Time points
    t_span = (0, t_max)
    t_eval = np.linspace(0, t_max, 1000)
    
    # Solve the ODE system
    solution = solve_ivp(
        ode_system, 
        t_span, 
        y0, 
        method='RK45', 
        t_eval=t_eval,
        rtol=1e-6,
        atol=1e-8
    )
    
    return solution

### Plot functions

In [ ]:
def plot_ode_solution(
    t_max=10.0, 
    X_init=28, XE_init=0.1, XP_init=0.2, XPE_init=0.01, XPP_init=0.02,
    E_init=0.01, E_step=10,
    k1=1, k2=0.1, k3=4, k4=1, k5=0.1, k6=4, k7=2, k8=2
):
    """
    Solves the ODE system and plots the results
    """
    # Solve the system
    solution = solve_signal_transduction_odes(
        t_max, X_init, XE_init, XP_init, XPE_init, XPP_init,
        E_init, E_step, k1, k2, k3, k4, k5, k6, k7, k8
    )
    
    t = solution.t
    X, XE, XP, XPE, XPP, E = solution.y
    
    # Create a figure with a specific size
    plt.figure(figsize=(12, 8))
    
    # Plot the results
    plt.subplot(2, 1, 1)
    plt.plot(t, X, label='X', linewidth=2)
    plt.plot(t, XE, label='XE', linewidth=2)
    plt.plot(t, XP, label='XP', linewidth=2)
    plt.plot(t, XPE, label='XPE', linewidth=2)
    plt.plot(t, XPP, label='XPP', linewidth=2)
    plt.axvline(x=1, color='gray', linestyle='--', alpha=0.7, label='E step change')
    plt.title('Signal Transduction System Dynamics')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot E separately since it has different magnitude
    plt.subplot(2, 1, 2)
    plt.plot(t, E, label='E', linewidth=2, color='green')
    plt.axvline(x=1, color='gray', linestyle='--', alpha=0.7)
    plt.title('Enzyme Concentration')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print the final values
    print(f"Final values at t={t_max}:")
    print(f"X: {X[-1]:.4f}")
    print(f"XE: {XE[-1]:.4f}")
    print(f"XP: {XP[-1]:.4f}")
    print(f"XPE: {XPE[-1]:.4f}")
    print(f"XPP: {XPP[-1]:.4f}")
    print(f"E: {E[-1]:.4f}")
    
    # Print mass conservation check
    total_initial = X_init + XE_init + XP_init + XPE_init + XPP_init
    total_final = X[-1] + XE[-1] + XP[-1] + XPE[-1] + XPP[-1]
    print(f"\nMass conservation check:")
    print(f"Total initial X species: {total_initial:.4f}")
    print(f"Total final X species: {total_final:.4f}")
    print(f"Difference: {total_final - total_initial:.8f}")

### Interactive cell

In [ ]:
# Create a more comprehensive interactive dashboard
def create_interactive_dashboard():
    # Initial condition widgets
    ic_widgets = [
        FloatSlider(min=0, max=50, step=0.5, value=28, description='X(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=1, step=0.01, value=0.1, description='XE(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=1, step=0.01, value=0.2, description='XP(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=0.1, step=0.001, value=0.01, description='XPE(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=0.1, step=0.001, value=0.02, description='XPP(0):', style={'description_width': '60px'})
    ]
    
    # Enzyme widgets
    enzyme_widgets = [
        FloatSlider(min=0, max=0.1, step=0.001, value=0.01, description='E(0):', style={'description_width': '60px'}),
        FloatSlider(min=0, max=20, step=0.1, value=10, description='E step:', style={'description_width': '60px'})
    ]
    
    # Kinetic parameter widgets
    k_widgets = [
        FloatSlider(min=0.1, max=5, step=0.1, value=1, description='k1:', style={'description_width': '40px'}),
        FloatSlider(min=0.01, max=1, step=0.01, value=0.1, description='k2:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=10, step=0.1, value=4, description='k3:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=5, step=0.1, value=1, description='k4:', style={'description_width': '40px'}),
        FloatSlider(min=0.01, max=1, step=0.01, value=0.1, description='k5:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=10, step=0.1, value=4, description='k6:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=5, step=0.1, value=2, description='k7:', style={'description_width': '40px'}),
        FloatSlider(min=0.1, max=5, step=0.1, value=2, description='k8:', style={'description_width': '40px'})
    ]
    
    # Time widgets
    time_widget = FloatSlider(min=5, max=50, step=1, value=10, description='Max Time:', style={'description_width': '80px'})
    
    # Output function for the interactive widgets
    out = interactive(plot_ode_solution, 
                      t_max=time_widget,
                      X_init=ic_widgets[0], XE_init=ic_widgets[1], XP_init=ic_widgets[2], 
                      XPE_init=ic_widgets[3], XPP_init=ic_widgets[4],
                      E_init=enzyme_widgets[0], E_step=enzyme_widgets[1],
                      k1=k_widgets[0], k2=k_widgets[1], k3=k_widgets[2], k4=k_widgets[3], 
                      k5=k_widgets[4], k6=k_widgets[5], k7=k_widgets[6], k8=k_widgets[7])
    
    # Create layout
    ic_box = VBox(ic_widgets, layout=Layout(border='1px solid #ddd', padding='10px', margin='5px'))
    enzyme_box = VBox(enzyme_widgets, layout=Layout(border='1px solid #ddd', padding='10px', margin='5px'))
    k_box = VBox(k_widgets, layout=Layout(border='1px solid #ddd', padding='10px', margin='5px'))
    
    # Create a reset button
    reset_button = Button(description='Reset Parameters', button_style='warning')
    
    def reset_params(b):
        ic_widgets[0].value = 28
        ic_widgets[1].value = 0.1
        ic_widgets[2].value = 0.2
        ic_widgets[3].value = 0.01
        ic_widgets[4].value = 0.02
        enzyme_widgets[0].value = 0.01
        enzyme_widgets[1].value = 10
        k_widgets[0].value = 1
        k_widgets[1].value = 0.1
        k_widgets[2].value = 4
        k_widgets[3].value = 1
        k_widgets[4].value = 0.1
        k_widgets[5].value = 4
        k_widgets[6].value = 2
        k_widgets[7].value = 2
        time_widget.value = 10
    
    reset_button.on_click(reset_params)
    
    # Top row - time widget and reset button
    top_row = HBox([time_widget, reset_button], layout=Layout(padding='10px'))
    
    # Middle row - initial conditions and enzyme parameters
    middle_row = HBox([
        VBox([
            ic_box,
            enzyme_box
        ]),
        k_box
    ])
    
    # Combine everything
    dashboard = VBox([
        top_row,
        middle_row,
        out.children[-1]  # The output area
    ])
    
    return dashboard

In [ ]:

# Run the interactive dashboard
dashboard = create_interactive_dashboard()
display(dashboard)

# Show the initial plot
plot_ode_solution()